# Trabalho CPA - Webscraping


### Integrantes: 
#### Andresssa Moreira ->  19204096-2
#### João Pedro de Moura -> matrícula

## Importação das bibliotecas


In [6]:
from bs4 import BeautifulSoup
import requests
import time
import re

In [7]:
def download(url, user_agent='wswp', num_retries=2, proxies=None):
    """ Download a given URL and return the page content
        args:
            url (str): URL
        kwargs:
            user_agent (str): user agent (default: wswp)
            proxies (dict): proxy dict w/ keys 'http' and 'https', values
                            are strs (i.e. 'http(s)://IP') (default: None)
            num_retries (int): # of retries if a 5xx error is seen (default: 2)
    """
    print('Downloading:', url)
    headers = {'User-Agent': user_agent}
    try:
        resp = requests.get(url, headers=headers, proxies=proxies)
        html = resp.text
        if resp.status_code >= 400:
            print('Download error:', resp.text)
            html = None
            if num_retries and 500 <= resp.status_code < 600:
                # recursively retry 5xx HTTP errors
                return download(url, num_retries - 1)
    except requests.exceptions.RequestException as e:
        print('Download error:', e)
        html = None
    return html

In [37]:
from tkinter import W


D = {}

for i in range(0, 26):
    url = "http://127.0.0.1:8000/places/default/index/" + str(i)

    f = requests.get(url)   

    soup = BeautifulSoup(f.text, "html.parser") 
    classRow = soup.find(class_ = "main row") # Pega a classe main row onde se encontra os países
    paisesLinks = classRow.find_all('a') # Pega a tag <a> onde contêm os links 

    lista = ['places_national_flag__row', 
                'places_area__row', 
                'places_population__row',
                'places_iso__row',
                'places_country__row',
                'places_capital__row',
                'places_continent__row',
                'places_tld__row',
                'places_currency_code__row',
                'places_currency_name__row',
                'places_phone__row',
                'places_postal_code_format__row',
                'places_postal_code_regex__row',
                'places_languages__row',
                'places_neighbours__row'
                ]

    for classRow in paisesLinks:

        pais = "http://127.0.0.1:8000" + classRow.get('href') 
        D[pais] = {}
        downloaded = download(pais) 
        
        for l in lista:
            soup = BeautifulSoup(downloaded, 'html.parser') 
            try: # esse try catch evita o erro de quando aparece esse link por exemplo: http://127.0.0.1:8000/places/default/index/1 (onde o find da um erro de atributo)
                target = soup.find(id = l)
                readonly = target.find(class_ ='readonly')
                w2p_fw = target.find(class_ = 'w2p_fw')

                readonly_regex = re.findall('([A-Za-z]*):\s<\/label>', str(readonly)) # type list
                w2p_fw_regex = re.findall('w2p_fw">(.*?)<\/td>', str(w2p_fw)) # type list

                # D[pais] = {D[pais][l]: [readonly, w2p_fw]}
                D[pais][l] = [readonly_regex, w2p_fw_regex]

                # eu não sei exatamente onde vai isso mas acho que ta certo     
                year, month, day, hour, minu = map(int, time.strftime("%Y %m %d %H %M").split())
                time_stamp = f'{day}/{month}/{year} {hour}:{minu}' 
                D[pais]['time_stamp'] = time_stamp

            except AttributeError:
                pass
    break
# D = {
#     'afeganistão': {'places_national_row': ['readonly', 'w2p_fw'], 'places_national_row': ['readonly', 'w2p_fw']},
#     'brasil': {'places_national_row': ['readonly', 'w2p_fw'], 'places_national_row': ['readonly', 'w2p_fw'],},
# }

Downloading: http://127.0.0.1:8000/places/default/view/Afghanistan-1
Downloading: http://127.0.0.1:8000/places/default/view/Aland-Islands-2
Downloading: http://127.0.0.1:8000/places/default/view/Albania-3
Downloading: http://127.0.0.1:8000/places/default/view/Algeria-4
Downloading: http://127.0.0.1:8000/places/default/view/American-Samoa-5
Downloading: http://127.0.0.1:8000/places/default/view/Andorra-6
Downloading: http://127.0.0.1:8000/places/default/view/Angola-7
Downloading: http://127.0.0.1:8000/places/default/view/Anguilla-8
Downloading: http://127.0.0.1:8000/places/default/view/Antarctica-9
Downloading: http://127.0.0.1:8000/places/default/view/Antigua-and-Barbuda-10
Downloading: http://127.0.0.1:8000/places/default/index/1


In [38]:
D

{'http://127.0.0.1:8000/places/default/view/Afghanistan-1': {'places_national_flag__row': [['Flag'],
   ['<img src="/places/static/images/flags/af.png"/>']],
  'time_stamp': '31/8/2022 13:39',
  'places_area__row': [['Area'], ['647500 square kilometres']],
  'places_population__row': [['Population'], ['29121286']],
  'places_iso__row': [['Iso'], ['AF']],
  'places_country__row': [['Country'], ['Afghanistan']],
  'places_capital__row': [['Capital'], ['Kabul']],
  'places_continent__row': [['Continent'],
   ['<a href="/places/default/continent/AS">AS</a>']],
  'places_tld__row': [['Tld'], ['.af']],
  'places_currency_code__row': [['Code'], ['AFN']],
  'places_currency_name__row': [['Name'], ['Afghani']],
  'places_phone__row': [['Phone'], ['93']],
  'places_postal_code_format__row': [['Format'], ['']],
  'places_postal_code_regex__row': [['Regex'], ['']],
  'places_languages__row': [['Languages'], ['fa-AF,ps,uz-AF,tk']],
  'places_neighbours__row': [['Neighbours'],
   ['<div><a href="/pl